In [1]:
import pandas as pd
import numpy as np
data =  pd.read_csv('datasets/titanic/train.csv')

In [2]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
a = pd.Series(name.split(',')[0] for name in data['Name'])
a.value_counts()

Andersson    9
Sage         7
Panula       6
Skoog        6
Carter       6
            ..
Hanna        1
Lewy         1
Mineff       1
Haas         1
Dooley       1
Name: count, Length: 667, dtype: int64

In [4]:
import matplotlib.pyplot as plt
plt.hist(data['Age'])

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
data.info()

In [ ]:
data.corrwith(data['Survived'])

In [ ]:
X = data.drop('Survived', axis = 1)

In [ ]:
y = data['Survived']

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.1, random_state = 42)
for train_index, test_index in splitter.split(data, data['Sex']):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y .iloc[test_index]

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def bucketize(feat):
    out = pd.cut(feat[:,0], bins = [i for i in range(int(feat.min()-1),int(feat.max()),int(feat.max()/9))] + [np.inf], labels = [i for i in range(1,11)])
    out = np.reshape(out, (out.shape[0],1))
    return(out)
normal_pipeline = make_pipeline(SimpleImputer(strategy = 'mean'), FunctionTransformer(bucketize, feature_names_out='one-to-one'), StandardScaler())
pipeline_2 = make_pipeline(StandardScaler(), FunctionTransformer(feature_names_out = 'one-to-one'))
pipeline_3 = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())
preprocessing = ColumnTransformer([('category', normal_pipeline, ['Age']), ('category2', normal_pipeline, ['Fare']), ('set2', pipeline_2, ['Pclass', 'SibSp', 'Parch']), 
                                  ('encode', pipeline_3, ['Sex', 'Embarked'])], remainder = 'drop')

In [ ]:
a = preprocessing.fit_transform(X_train)

In [ ]:
a = pd.DataFrame(a, columns = preprocessing.get_feature_names_out())


In [ ]:
X_train.iloc[a[a['category2__Fare'].isna() ==  True].index]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNC = KNeighborsClassifier(n_neighbors = 4)
KNC.fit(a, y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
recall_score(y_train, cross_val_predict(KNC, a, y_train, cv=4))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(KNC, a, y_train, cv = 4, scoring = 'accuracy')

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier()
cross_val_score(dummy, a, y_train, cv = 4, scoring = 'accuracy')

In [ ]:
from sklearn.model_selection import cross_val_predict
pred1 = cross_val_predict(KNC, a, y_train, cv=4)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, pred)
cm

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.ensemble  import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(a, y_train)

In [ ]:
cross_val_score(rfc, a, y_train, cv=4, scoring='precision')

In [ ]:
pred = cross_val_predict(KNC, a, y_train, cv=4, method = 'predict_proba')
scores = pred[:, 1]
pred1 = cross_val_predict(KNC, a, y_train, cv=4, method = 'predict_proba')
scores1 = pred[:, 1]

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
fpr, tpr, threshold = roc_curve(y_train, scores1)
prec, rec, thresh = precision_recall_curve(y_train, scores1)
idx = (rec <= 0.90).argmax()
thresh_90 = thresh[idx]
plt.plot(fpr, tpr, linewidth = 2, label = 'roc_curve')

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, scores)

In [ ]:
thresh_90